In [28]:
from __future__ import absolute_import, division, print_function

from os import listdir
import xmltodict
import pandas
from StringIO import StringIO
from tqdm import tqdm
import math
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
from datetime import date
from praelexis_file_system_provider.data_access_layer import AzureBlobStorageDataAccessLayer
import PyPDF2
import subprocess
import shutil
import os

from PIL import Image
import commands
import time
import glob
from shutil import copyfile
import os.path
import commands
import time

%matplotlib inline

### Connect to Blob Store
Opens up a connection to the Azure blog store where the PDF and XML files provided by Capitec are stored.

In [29]:
blob_dal = AzureBlobStorageDataAccessLayer('devcapitecdata', 'Gt5z0EAAAm5ZETk//WBnRiQUZ9FDB/BmLmbx5ZPRtDwh6vxTDAB/XzLlM+m/epfnDYcZ8JDfMEGg1p9tBW0iFQ==', 'bank-statements', home_directory='drop/Documentum_line_files')
blob_dal.connect()
files = blob_dal.list(path='CI')

print ("Number of files: ", len(files))

Number of files:  8000


### Pdf viewer

#### Functions

In [66]:
def how_many_files(src_dir):
    """
    Notifies the user as to how many example files there are in the directory.
    """
    max_example = 0 

    # How many text files we have in the source dir
    for i in range(0, len(files)):
        # file name 
        loc_dir = src_dir + "Example" + str(i + 1) + "_loc.txt"

        # if the file does not exist, we have reached the limit
        if not os.path.isfile(loc_dir): 
            max_example = i-1
            print("Up to example " + str(i-1) + " has successfully been calculated.")
            print("Only view examples lower or equal to Example " + str(i-1) + ".")
            break
            
    return max_example

def download_pdf(example_number, output_name, src_dir, working_dir):
    """
    Downloads the PDF from the source dir and saves it to the local working dir
    """
    # Select the pdf wanted from src_dir
    loc_dir = src_dir + "Example" + str(example_number)+"_loc.txt"
    f = open(loc_dir,'r')
    pdf_src_dir = f.readline()
    # Create a temp pdf file in this working dir
    stream = StringIO()
    blob_dal.get(pdf_src_dir, stream)

    pdf_dst_dir = working_dir +output_name + ".pdf"

    with open (pdf_dst_dir, 'w') as tmp:
        stream.seek(0)
        shutil.copyfileobj(stream, tmp)
        
def create_tif(output_name, working_dir):
    """
    Creates a tif file in the working dir
    """
    tif_dir = working_dir + output_name + ".tif"
    pdf_dst_dir = working_dir + output_name + ".pdf"
    if os.path.isfile(pdf_dst_dir):
        start = time.time()
        print("Creating the tif file...")
        s=commands.getstatusoutput("convert -density 300 -units pixelsperinch " + pdf_dst_dir + " -quality 100 " + tif_dir)
        print("It took ",round(time.time()-start,2)," seconds to convert the pdf to tif.")
    else:
        print("The pdf has not been created yet. Please create the pdf first.")
def copy_txt(example_number,output_name,src_dir,working_dir):
    txt_src_dir = src_dir + "Example" + str(example_number)+".txt"
    txt_dst_dir = working_dir + output_name + ".txt"
    copyfile(txt_src_dir, txt_dst_dir)
    
def copy_box(example_number,output_name,src_dir,working_dir):
    box_src_dir = src_dir + "Example" + str(example_number)+".box"
    box_dst_dir = working_dir + output_name + ".box"
    copyfile(box_src_dir, box_dst_dir)

#### Main Code

In [67]:
# Set the example number to view
example_number = 2000
output_name = "DriesOutput"

src_dir = "./test_files/"
working_dir = "./view_output/"

max_example = how_many_files(src_dir=src_dir)

# Error handeling
if example_number > max_example:
    print("This example number has not been generated yet. Setting the example number to the max value of:", max_example)
    example_number = max_example

Up to example 1876 has successfully been calculated.
Only view examples lower or equal to Example 1876.
This example number has not been generated yet. Setting the example number to the max value of: 1876


In [74]:
# Download the given example pdf
download_pdf(example_number, output_name, src_dir, working_dir)
print("Pdf created.")

Pdf created.


In [78]:
# Create a output tif file for the given pdf
create_tif(output_name, working_dir)
print("Tif file created.")

Creating the tif file...
It took  8.44  seconds to convert the pdf to tif.
Tif file created.


In [76]:
# Copy the text file for the given example
copy_txt(example_number,output_name,src_dir,working_dir)
print("Text file copied.")

Text file copied.


In [75]:
# Copy the box file for the given example
copy_box(example_number,output_name,src_dir,working_dir)
print("Box file copied.")

Box file copied.
